In [2]:
import requests
import pandas as pd
from collections import defaultdict
from decimal import Decimal

# URL de la API local para ventas y productos (ajusta las rutas y puertos según tu configuración)
url_ventas = 'http://localhost:3001/ventas'
url_productos = 'http://localhost:3001/productos'

# Realizar solicitudes a las APIs
response_ventas = requests.get(url_ventas)
response_productos = requests.get(url_productos)

# Verificar el estado de las respuestas
if response_ventas.status_code == 200 and response_productos.status_code == 200:
    # Convertir las respuestas a JSON
    data_ventas = response_ventas.json()
    data_productos = response_productos.json()

    # Crear DataFrames con los datos
    df_ventas = pd.DataFrame(data_ventas)
    df_productos = pd.DataFrame(data_productos)

    # Inicializar una variable para almacenar la suma total de ventas
    suma_total_ventas = Decimal(0)

    # Supongamos que las API devuelven listas de ventas y productos con campos "id_producto", "cantidad" y "precio"
    for venta in data_ventas:
        id_producto = venta.get('id_producto', 0)
        cantidad_venta = int(venta.get('cantidad', 0))
        precio_producto = Decimal([producto['precio'] for producto in data_productos if producto['id_producto'] == id_producto][0])
        total_venta = cantidad_venta * precio_producto

        # Acumular la venta en la suma total
        suma_total_ventas += total_venta

    # Convertir el total de ventas a un número entero
    suma_total_ventas = int(suma_total_ventas)

    # Encontrar la categoría más vendida
    id_categoria_mas_vendida = df_productos['id_producto'].value_counts().idxmax()
    nombre_categoria_mas_vendida = df_productos.loc[df_productos['id_producto'] == id_categoria_mas_vendida, 'nombre_categoria'].values[0]

    # Encontrar el producto más vendido
    productos_vendidos = defaultdict(int)
    for venta in data_ventas:
        producto = venta['nombre_producto']
        cantidad_vendida = venta['cantidad']
        productos_vendidos[producto] += cantidad_vendida

    producto_mas_vendido = max(productos_vendidos, key=productos_vendidos.get)
    cantidad_mas_vendida = productos_vendidos[producto_mas_vendido]

    # Mostrar resultados
    print(f'Ingreso total de todas las ventas: $ {suma_total_ventas} pesos')
    print("Categoría más vendida:", nombre_categoria_mas_vendida)
    print(f'El producto más vendido es "{producto_mas_vendido}" con {cantidad_mas_vendida} unidades vendidas.')

    # Guardar los resultados en un archivo CSV
    resultados = {
        'Ingreso Total': [suma_total_ventas],
        'Categoría Más Vendida': [nombre_categoria_mas_vendida],
        'Producto Más Vendido': [f'{producto_mas_vendido} ({cantidad_mas_vendida} unidades vendidas)']
    }

    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv('Exel_ventas.csv', index=False)
else:
    print("Error al obtener datos de las APIs")

Ingreso total de todas las ventas: $ 489226 pesos
Categoría más vendida: Playeras
El producto más vendido es "Tenis53A" con 100 unidades vendidas.
